# **Statistics for Angle to ablation timelapse**

*(A2A_Tmlps_Stats) by Stephane Verger*

This notebook was made to propose a simple and reproducible way to plot and analyze the data from the "MT Angle2Ablation workflow".

It can be used with no prior knowledge of python programming. However it can be useful to acquire a little bit of background on how to use this tool. We recommand the very useful lecture series on BioImage Analysis from Robert Haase (https://www.youtube.com/playlist?list=PL5ESQNfM5lc7SAMstEu082ivW4BDMvd0U), in particular the lectures 9 to 11 about the use of python for data analysis and hypothesis testing.

## **How to run the code?**

**To run a cell containing code simply select it and hit enter+shift. Run each cell one by one while carefully following the instructions between each cell**

Briefly, there are 4 steps:
1. Load the required packages and functions.
2. Upload the data.
3. Generate the table and graph output with interactive widgets.
4. Save the desired output.

**1. Loading libraries**

Here, simply select the cell below, hit enter+shift to run the cell (load the packages) and continue to the next step.

In [ ]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, fixed
from IPython.display import display
from ipyfilechooser import FileChooser

**2. Data upload**
1. Run the cell below.
2. Then, click on the "Upload" button and select the file to be analyzed.
3. Load the data to the notebook, by **running either** the cell called "#For Binder" (if you are running the notebook on binder) or "#For local install" if you are running it on you machine.

In [ ]:
#Widget to upload data
uploader = widgets.FileUpload(
    accept='.txt',
    multiple=False
)
display(uploader)

For binder

In [ ]:
#For binder
#Data upload and read
uploaded_file = list(uploader.value)[0]
all_data = pd.read_csv(io.BytesIO(uploaded_file['content']), sep = "\t")

For local install

In [ ]:
#For local install
#Data upload and read
uploaded_file = list(uploader.value.values())[0]
all_data = pd.read_csv(io.BytesIO(uploaded_file['content']), sep = "\t")

**3. Graphs and tables**
1. Run the cell below.
2. Then scroll down and to see the output and widget interaction.

You can select which genotype data are displayed (Click + shift or ctrl to select multiple), whether average per genotype or samples, or individual cells are displayed, and whether to show the output for the angle or anisotropy values.

In [ ]:

#Message
print('Warning: It is not possible to select "show individual samples" = No and "Show induvidual cells" = Yes at the same time.')
print('Currently the table generation only supports the selection of multiple genotypes when "show individual samples" and "cells" = No.')
print('Alternatively the "individual sample" data table can be shown if only one genotype is selected at a time.')
print('\nClick + shift or ctrl or command to select multiple genotypes.')
print('To change and fix the x axis limit of the plot to a defined value, select a value different from zero in the x axis limit widget. Make sure that both up and low values are set in that case.') 

#Widgets
#Get groups of genotypes/conditions
GenoGroup = all_data.groupby(['Genotype/condition'])
GenoGroupKeys = list(GenoGroup.groups.keys())
#style for menus
style = {'description_width': 'initial'}
#Widget select multiple menu to choose genotype to plot, toggle to show individual samples, cells and choose between angle and anisotropy
SelectGeno= widgets.SelectMultiple(options=GenoGroupKeys, value=[GenoGroupKeys[0]], description='Genotype/condition to plot.', disabled=False, style=style, rows=len(GenoGroupKeys), layout={'width': 'max-content'})
ShowSamples = widgets.ToggleButtons(options=[('No', False), ('Yes', 'Genotype/condition')], value=False, description='Show individual samples', button_style='', style=style)
ShowCells = widgets.ToggleButtons(options=[('No', False), ('Yes', 'Sample Number')], value=False, description='Show individual cells', button_style='', style=style)
ChooseValue = widgets.ToggleButtons(options=[('Angle to ablation', 'Acute absolute angle to ablation'), ('Anisotropy', 'Anisotropy')], value='Acute absolute angle to ablation', description='Value to plot', button_style='', style=style)
ChooseXlimUp = widgets.FloatText(value=0, description='X axis limit up', disabled=False, style=style)
ChooseXlimLow = widgets.FloatText(value=0, description='X axis limit low', disabled=False, style=style)


#Function to define the timelapse graph
def datagraph(whichdata, col, row, ydata, XlimUp, XlimLow, save, path):
    

    #subsets data to be shown in plot
    subdata = all_data[all_data['Genotype/condition'].isin(whichdata)]
    #Display more or less details from genotype to individual cells
    if (ShowSamples.value==False and ShowCells.value==False):
        huedata = 'Genotype/condition'
        tograph = True
        totable = True
    elif (ShowSamples.value=='Genotype/condition' and ShowCells.value==False):
        huedata = 'Sample Number'
        tograph = True
        totable = True
    elif (ShowSamples.value=='Genotype/condition' and ShowCells.value=='Sample Number'):
        huedata = 'CellNumber'
        tograph = True
        totable = False
    elif (ShowSamples.value==False and ShowCells.value=='Sample Number'):
        huedata = 'CellNumber'
        tograph = False
        totable = False
    if (len(SelectGeno.value) > 1 and ShowSamples.value=='Genotype/condition'):
        totable = False
    #plot
    if (tograph==True):
        fig = sns.relplot(data=subdata, col=col, row=row,  x="Timepoint", y=ydata, hue=huedata, errorbar=('ci', 95), kind="line")
        if (XlimLow>0 or XlimUp>0):
            fig.set(ylim=(XlimLow, XlimUp))
    
    #Table
    if (totable==True):
        #subsets data to be shown in table
        SubGenoTimeGroup = subdata.groupby([huedata, 'Timepoint'], as_index=False)
        #Aggregate with mean and Bootstraped 95 CI
        MeanSGTG = SubGenoTimeGroup[ydata].agg(np.mean).round(3)
        BtstrpSGTG = SubGenoTimeGroup[ydata].agg(lambda x: sns.utils.ci(sns.algorithms.bootstrap(x, func=np.mean, n_boot=1000, units=None))[1]-(np.mean(x))).round(3)
        #Pivot, merged and display timeseries table
        pivMeanSGTG = MeanSGTG.pivot(index=huedata, columns='Timepoint', values=ydata)
        pivBtstrpSGTG = BtstrpSGTG.pivot(index=huedata, columns='Timepoint', values=ydata)
        Finaltable = pivMeanSGTG.astype('string') + ' +- ' + pivBtstrpSGTG.astype('string')
        print('\n\n The data displayed in both table and graph is the mean values +- bootstrapped 95% confidence interval')
        display(Finaltable) 
    
    #Save table and plot
    if (save == True):
        if (totable==True):
            Finaltable.to_csv(path + '.csv')
        if (tograph==True):
            plt.savefig(path + '.svg')
    
#Interact widget
DataGraphWidget = widgets.interactive(datagraph, whichdata=SelectGeno, col=ShowSamples, row=ShowCells, ydata=ChooseValue, XlimUp = ChooseXlimUp, XlimLow = ChooseXlimLow, save = fixed(False), path = fixed(False));
DataGraphWidget


**4. Save data and graphs**
1. Run the next two cells below.
2. Click on the "Select" button and select a destination folder as well as an output file name (e.g. GFP-MBD_Angles).
Avoid spaces and special characters in the name.
3. Validate by clicking "Select" again. This should display in green the destination path with file name.
4. Finally, run the last cell. This will directly save the table as a .csv file and the graph as a .svg file with the choosen file name in the selected destination folder.

Note 1: When running the notebook on binder, the data can only be saved on the temporary binder folder that is created to run the notebook online. In the cell where you choose the path for file saving, navigate to /home/Jovyan/. The data saved in the folder will apprear in the file browser on the left of the notebook on this page. You can double click on it to open it in a new window here, or right click and select download to download it on your computer. 

Note 2: You can go back and forth between the interactive widgets and display above, and this save step, but you will have to reload the first cell below everytime before saving to get the current status of the table and graphs diplayed, and you may need to change the output file name, before running the last cell to save.

Note 3: The .csv file can be opened in any text and spreadsheet editor. The .svg file can be opened and further modified (e.g. to change the time point naming) with any vector graphic editor (e.g. Inkscape or Illustrator). 

In [ ]:
#Retrieve selected value from interactive widget
whichdata = DataGraphWidget.children[0].value
col = DataGraphWidget.children[1].value
row = DataGraphWidget.children[2].value
ydata = DataGraphWidget.children[3].value
XlimUp = DataGraphWidget.children[4].value
XlimLow = DataGraphWidget.children[5].value

In [ ]:
#FileChooser
starting_directory = '/' 
chooser = FileChooser(starting_directory)

#Display
print('Suggested output filename content: ' + str(whichdata) + ', ' + str(ydata))
print('On binder, choose file path /home/jovyan/ and dowload you data afterwards from binder.')
display(chooser)

In [ ]:
print ('Saved table below as .csv file and graph as .svg file')
Path = chooser.selected
datagraph(whichdata, col, row, ydata, XlimUp, XlimLow, True, Path)